In [ ]:
import csv
import json
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import numpy as np
import seaborn as sns

In [ ]:
rt_reviews_df = pd.read_csv('../Data/rt.reviews.tsv',
                           delimiter='\t',
                           encoding=('unicode_escape'))

In [ ]:
rt_reviews_df.head()

In [ ]:
rt_reviews_df['rating'].value_counts()

In [ ]:
tnmovie = pd.read_csv('../Data/tn.movie_budgets.csv')

In [ ]:
tnmovie['worldwide_gross'] = tnmovie['worldwide_gross'].str.replace("\$|,|\.", '', regex = True).astype('int64')

tnmovie['domestic_gross'] = tnmovie['domestic_gross'].str.replace("\$|,|\.", '', regex = True).astype('int64')

tnmovie['production_budget'] = tnmovie['production_budget'].str.replace("\$|,|\.", '', regex = True).astype('int64')

tnmovie['profit'] = tnmovie['worldwide_gross'] - tnmovie['production_budget']

tnmovie['perc_return'] = (tnmovie['profit'] / tnmovie['production_budget']) * 100

# Reading data

Read the im.db data with SQL

In [ ]:
conn = sqlite3.connect('../Data/im.db')
pd.read_sql('''
SELECT
    *
FROM
    sqlite_master

''',conn)

# Directors SQL

Use SQL query to join tables by primary and foreign keys. This new comprehensive dataframe contains indentifying data regarding directors.

In [ ]:
directors_df = pd.read_sql('''
SELECT
    category,
    pr.person_id,
    pr.movie_id,
    primary_name,
    primary_title
FROM
    principals as pr
INNER JOIN
    persons as p using(person_id)
INNER JOIN
    movie_basics as mb using(movie_id)
WHERE
    category = 'director'
''',conn)

directors_df

# Directors dataframe merge

Merge the directors and tnmovie dataframes on 'primary_title' and 'movie'

In [ ]:
directors_df = pd.merge(directors_df,
                  tnmovie,
                  left_on='primary_title',
                  right_on='movie')

directors_df

# Top Directors by Profit

Count the values associated with the 15 most common directors that appear in the sorted directors df.

In [ ]:
top_15_dirs = directors_df.value_counts('primary_name').head(15)

top_15_dirs

Make a list of director names using index.

In [ ]:
top_15_dirs_list = list(top_15_dirs.index)

top_15_dirs_list

Iterate through the list of top 15 directors.

Next, locate the rows and columns where the 'primary_name' is equal to the director's name.

Then, append the average profit for each movie the director directed in to a new list.

In [ ]:
avg_dir = []

for director in top_15_dirs_list:
    directors_filtered = directors_df.loc[directors_df['primary_name'] == director]
    avg_dir.append(directors_filtered.profit.mean())
    
avg_dir

Create a dataframe using zip with the top 15 directors and average profit lists

In [ ]:
directing_df = pd.DataFrame(list(zip(top_15_dirs_list, avg_dir)),
               columns =['director_name', 'average_profit'])

Sort the dataframe by 'average_profit'

In [ ]:
directing_df.sort_values(by='average_profit', inplace=True, ascending=False)

# Directors data visualization

In [ ]:
#fig, ax = plt.subplots(figsize=(14,9))

#x = rev_dir
#y = top_20_dirs

#ax.set_title("Top 20 Directors in Highest Earning Movies")
#ax.set_ylabel("Directors")
#ax.set_xlabel("Average Profit Per Movie")

#ax.ticklabel_format(axis='x', style='plain')

#ax.barh(width=x, y=y);

Create a barh using seaborn with the average profit for each movie the director directed on the x-axis and the top 15 directors list on the y-axis. 

Change x-axis units from scientific notation to easily readable numbers.

In [ ]:
sns.set(style="whitegrid")

sns.color_palette("rocket")

sns.set(rc = {'figure.figsize':(15,10)})

g = sns.barplot(x='average_profit', y='director_name', data=directing_df.head(5))

plt.ticklabel_format(axis='x', style='plain')
plt.title('Top 5 Directors in Highest Grossing Movies', fontsize=30)
plt.xlabel('Average Profit Per Movie', fontsize=24)
plt.ylabel('Directors', fontsize=24)

ticks_loc = g.get_xticks().tolist()
#g.xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))

#g.set_xticklabels([label_format.format(x) for x in ticks_loc])

xlabels = ['{:,.0f}'.format(x) + ' MM' for x in g.get_xticks()/1000000]
g.set_xticklabels(xlabels);

# Insights

Directors draw in audiences! Steven Spielbergs, Tim Burtons, and Ridley Scotts 
create universes that pull over $100 million in box office. These directors have unqiue, 
beloved fimlamking styles. And their fans come out in droves to experience them in theaters. 
Sometimes multiple times. With one these visionaries at the helm, Microsoft may likley produce 
an instant classic and a blockbuster hit at the same time.

# Actors SQL

Use SQL query to join tables by primary and foreign keys. This new comprehensive dataframe contains indentifying data regarding actors.

In [ ]:
actors = pd.read_sql('''
SELECT
    category,
    pr.person_id,
    pr.movie_id,
    primary_name,
    primary_title
FROM
    principals as pr
INNER JOIN
    persons as p using(person_id)
INNER JOIN
    movie_basics as mb using(movie_id)
WHERE
    category = 'actor' 
    or
    category = 'actress'
''',conn)

actors

# Actors dataframe merge

Merge the actors and tnmovie dataframes on 'primary_title' and 'movie'

In [ ]:
actors_df = pd.merge(actors,
                  tnmovie,
                  left_on='primary_title',
                  right_on='movie')

actors_df

# Top Actors by Profit

Count the values associated with the 15 most common names that appear in the sorted actors df.

In [ ]:
top_15_actors = actors_df.value_counts('primary_name').head(15)

top_15_actors

Make a list of actor names using index.

In [ ]:
top_15_actors = list(top_15_actors.index)

top_15_actors

Iterate through the list of top 15 actors.

Next, locate the rows and columns where the 'primary_name' is equal to the actors's name.

Then, append the average profit for each movie the actor appeared in to a new list.

In [ ]:
avg_actor = []

for actor in top_15_actors:
    actors_filtered = actors_df.loc[actors_df['primary_name'] == actor]
    avg_actor.append(actors_filtered.profit.mean())
    
avg_actor[0]

Create a dataframe using zip with the top 15 actors and average profit lists

In [ ]:
acting_df = pd.DataFrame(list(zip(top_15_actors, avg_actor)),
               columns =['actor_name', 'average_profit'])

Sort the dataframe by 'average_profit'

In [ ]:
acting_df.sort_values(by='average_profit', inplace=True, ascending=False)

# Actors data visualization

In [ ]:
#fig, ax = plt.subplots(figsize=(14,9))


#x = rev_actor
#y = top_25_actors

#ax.set_title("Top 25 Actors in Highest Earning Movies")
#ax.set_ylabel("Actors")
#ax.set_xlabel("Average Profit Per Movie")

#ax.ticklabel_format(axis='x', style='plain')

#xlabels = ['{:,.0f}'.format(x) + ' MM' for x in ax.get_xticks()/1000000]
#ax.set_xticklabels(xlabels)

#ax.barh(width=x, y=y);

Create a barh using seaborn with the average profit for each movie the actor appeared in on the x-axis and the top 15 actors list on the y-axis.

Change x-axis units from scientific notation to easily readable numbers.

In [ ]:
sns.set(style="whitegrid")

sns.color_palette("rocket")

sns.set(rc = {'figure.figsize':(15,10)})

g = sns.barplot(x='average_profit', y='actor_name', data=acting_df.head(5))

plt.title('Top 5 Actors in Highest Grossing Movies', fontsize=30)
plt.xlabel('Average Profit Per Movie', fontsize=24)
plt.ylabel('Actors', fontsize=24)

plt.ticklabel_format(axis='x', style='plain')

xlabels = ['{:,.0f}'.format(x) + ' MM' for x in g.get_xticks()/1000000]
g.set_xticklabels(xlabels);

# Insights

Movie star appeal is a major box office draw! Actors like Dwayne Johnson, Jennifer Lawrence, and Kevin Hart not only makes us laugh and cry, they also inspire us. These actors average over $150 million per film because they have an army of die-hard fans always ready to invade the next midnight premiere. Casting one these stars in a Microsoft movie, may likley drive high box office sales.